### **Thanks to great works from Prem, Jasonczh, Chris, Jasonczh, Giba, Nin7a1, judith007,samu2505, tetsutani.**

## **This notebook mainly tuned the models and explain the reasons of doing these things. Hope my words can help you in this competition!**

In [1]:
import gc
import numpy as np
import pandas as pd
import os
from tqdm.notebook import tqdm
BASE = './12/'

def smape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * np.mean(smap)

def vsmape(y_true, y_pred):
    smap = np.zeros(len(y_true))
    
    num = np.abs(y_true - y_pred)
    dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
    
    pos_ind = (y_true!=0)|(y_pred!=0)
    smap[pos_ind] = num[pos_ind] / dem[pos_ind]
    
    return 100 * smap

In [2]:
census = pd.read_csv(BASE + 'census_starter.csv')
train = pd.read_csv(BASE + 'train.csv')
reaveal_test = pd.read_csv(BASE + 'revealed_test.csv')
train = pd.concat([train, reaveal_test]).sort_values(by=['cfips','first_day_of_month']).reset_index()
test = pd.read_csv(BASE + 'test.csv')
drop_index = (test.first_day_of_month == '2022-11-01') | (test.first_day_of_month == '2022-12-01')
test = test.loc[~drop_index,:]
sub = pd.read_csv(BASE + 'sample_submission.csv')
coords = pd.read_csv(BASE + "cfips_location.csv")
print(train.shape, test.shape, sub.shape)

train['istest'] = 0
test['istest'] = 1
raw = pd.concat((train, test)).sort_values(['cfips','row_id']).reset_index(drop=True)
raw = raw.merge(coords.drop("name", axis=1), on="cfips")

raw['state_i1'] = raw['state'].astype('category')
raw['county_i1'] = raw['county'].astype('category')
raw['first_day_of_month'] = pd.to_datetime(raw["first_day_of_month"])
raw['county'] = raw.groupby('cfips')['county'].ffill()
raw['state'] = raw.groupby('cfips')['state'].ffill()
raw["dcount"] = raw.groupby(['cfips'])['row_id'].cumcount()
raw['county_i'] = (raw['county'] + raw['state']).factorize()[0]
raw['state_i'] = raw['state'].factorize()[0]
raw['scale'] = (raw['first_day_of_month'] - raw['first_day_of_month'].min()).dt.days
raw['scale'] = raw['scale'].factorize()[0]
os.environ["CUDA_VISIBLE_DEVICES"]="0"


(128535, 8) (18810, 3) (25080, 2)


# There are some anomalies, specially at timestep 18

In [3]:
for o in tqdm(raw.cfips.unique()): 
    indices = (raw['cfips'] == o)  
    tmp = raw.loc[indices].copy().reset_index(drop=True)  
    
    var = tmp.microbusiness_density.values.copy()
    
    for i in range(37, 2, -1):
        
        thr = 0.10 * np.mean(var[:i]) 
        difa = var[i] - var[i - 1] 
        if (difa >= thr) or (difa <= -thr):  
            if difa > 0:
                var[:i] += difa - 0.003 
            else:
                var[:i] += difa + 0.003 
    var[0] = var[1] * 0.99
    raw.loc[indices, 'microbusiness_density'] = var


  0%|          | 0/3135 [00:00<?, ?it/s]

In [4]:
lag = 1
raw[f'mbd_lag_{lag}'] = raw.groupby('cfips')['microbusiness_density'].shift(lag).bfill()
raw['dif'] = (raw['microbusiness_density'] / raw[f'mbd_lag_{lag}']).fillna(1).clip(0, None) - 1
raw.loc[(raw[f'mbd_lag_{lag}']==0), 'dif'] = 0
raw.loc[(raw[f'microbusiness_density']>0) & (raw[f'mbd_lag_{lag}']==0), 'dif'] = 1
raw['dif'] = raw['dif'].abs()
# raw.groupby('dcount')['dif'].sum().plot()

# SMAPE is a relative metric so target must be converted.

In [5]:
raw['target'] = raw.groupby('cfips')['microbusiness_density'].shift(-1)
raw['target'] = raw['target']/raw['microbusiness_density'] - 1

In [6]:
raw['lastactive'] = raw.groupby('cfips')['active'].transform('last')

# dt = raw.loc[raw.dcount==40].groupby('cfips')['microbusiness_density'].agg('last')
# raw['lastactive'].clip(0, 8000).hist(bins=30)

# Feature Engineering
> Try tuning this part

In [7]:
def build_features(raw, target='microbusiness_density', target_act='active_tmp', lags = 6):
    feats = []   

    for lag in range(1, lags):
        raw[f'mbd_lag_{lag}'] = raw.groupby('cfips')[target].shift(lag)
        raw[f'act_lag_{lag}'] = raw.groupby('cfips')[target_act].diff(lag)
        feats.append(f'mbd_lag_{lag}')
        feats.append(f'act_lag_{lag}')
        
    lag = 1
    for window in [2, 4, 6, 8, 10]:
        raw[f'mbd_rollmea{window}_{lag}'] = raw.groupby('cfips')[f'mbd_lag_{lag}'].transform(lambda s: s.rolling(window, min_periods=1).sum())        
        feats.append(f'mbd_rollmea{window}_{lag}')
    
    census_columns = list(census.columns)
    census_columns.remove( "cfips")
    
    raw = raw.merge(census, on="cfips", how="left")
    feats += census_columns
    
    co_est = pd.read_csv("./12/co-est2021-alldata.csv", encoding='latin-1')
    co_est["cfips"] = co_est.STATE*1000 + co_est.COUNTY
    co_columns = [
        'SUMLEV',
        'DIVISION',
        'ESTIMATESBASE2020',
        'POPESTIMATE2020',
        'POPESTIMATE2021',
        'NPOPCHG2020',
        'NPOPCHG2021',
        'BIRTHS2020',
        'BIRTHS2021',
        'DEATHS2020',
        'DEATHS2021',
        'NATURALCHG2020',
        'NATURALCHG2021',
        'INTERNATIONALMIG2020',
        'INTERNATIONALMIG2021',
        'DOMESTICMIG2020',
        'DOMESTICMIG2021',
        'NETMIG2020',
        'NETMIG2021',
        'RESIDUAL2020',
        'RESIDUAL2021',
        'GQESTIMATESBASE2020',
        'GQESTIMATES2020',
        'GQESTIMATES2021',
        'RBIRTH2021',
        'RDEATH2021',
        'RNATURALCHG2021',
        'RINTERNATIONALMIG2021',
        'RDOMESTICMIG2021',
        'RNETMIG2021'
    ]
    raw = raw.merge(co_est, on="cfips", how="left")
    feats +=  co_columns
    return raw, feats

In [9]:
values_list = [13101.0,48301.0,13265.0,28055.0,48033.0,48261.0,48269.0,31005.0,31009.0,31115.0,31165.0,48393.0,16033.0,31183.0,38087.0,2282.0,31007.0,31113.0,46003.0,46017.0]
Kmean_df = pd.read_csv('./' + 'k_means_graph_scaled_update.csv')
#Kmean_50 = pd.read_csv('./' + 'k_means_graph_50.csv')
#kmeans=Kmean_50[['cfips','K-means_Elbow']]
#kmeans.columns = ['cfips','Elbow']

In [10]:
raw = raw.merge(Kmean_df[['cfips','K-means_Silhouette','K-means_Elbow']], on="cfips", how="left")
#raw = raw.merge(kmeans[['cfips','Elbow']], on="cfips", how="left")

In [13]:
# Build Features based in lag of target
raw, feats = build_features(raw, 'target', 'active', lags = 8)
features = ['state_i']
features += feats
features += ['lng','lat','scale']
# print(features)
# raw.loc[raw.dcount==40, features].head(10)
features += ['K-means_Silhouette','K-means_Elbow']
features.remove('state_i')

Latitude and Longitude feature engineering from samu2505.

In [14]:
coordinates = raw[['lng', 'lat']].values

# Encoding tricks
emb_size = 20
precision = 1e6

latlon = np.expand_dims(coordinates, axis=-1)

m = np.exp(np.log(precision)/emb_size)
angle_freq = m ** np.arange(emb_size)
angle_freq = angle_freq.reshape(1,1, emb_size)
latlon = latlon * angle_freq
latlon[..., 0::2] = np.cos(latlon[..., 0::2])

In [15]:
def rot(df):
    for angle in [15, 30, 45]:
        df[f'rot_{angle}_x'] = (np.cos(np.radians(angle)) * df['lat']) + \
                                (np.sin(np.radians(angle)) * df['lng'])
        
        df[f'rot_{angle}_y'] = (np.cos(np.radians(angle)) * df['lat']) - \
                                (np.sin(np.radians(angle)) * df['lng'])
        
    return df

raw = rot(raw)

In [16]:
features += ['rot_15_x', 'rot_15_y', 'rot_30_x', 'rot_30_y', 'rot_45_x', 'rot_45_y']

In [17]:
def get_model():
    from sklearn.ensemble import VotingRegressor
    import lightgbm as lgb
    import xgboost as xgb
    import catboost as cat
    from sklearn.pipeline import Pipeline
    from sklearn.neighbors import KNeighborsRegressor
    from sklearn.impute import KNNImputer    

# we should decrease the num_iterations of catboost
    cat_model = cat.CatBoostRegressor(
        iterations=800,
        loss_function="MAPE",
        verbose=0,
        grow_policy='SymmetricTree',
        learning_rate=0.035,
        colsample_bylevel=0.8,
        max_depth=5,
        l2_leaf_reg=0.2,
#         max_leaves = 17,
        subsample=0.70,
        max_bin=4096,
    )

    return cat_model
    

In [18]:
ACT_THR = 140
MONTH_1 = 39
MONTH_last = 40

In [19]:
raw['ypred_last'] = np.nan
raw['ypred'] = np.nan
raw['k'] = 1.
raw['microbusiness_density'].fillna(0, inplace = True)

for TS in range(MONTH_1, MONTH_last): #40):
    print(TS)

    model = get_model()
            
    train_indices = (raw.istest==0) & (raw.dcount  < TS) & (raw.dcount >= 19) & (raw.lastactive>ACT_THR) & (~raw['cfips'].isin(values_list))
    valid_indices = (raw.istest==0) & (raw.dcount == TS) 
    model.fit(
        raw.loc[train_indices, features],
        raw.loc[train_indices, 'target'].clip(-0.0043, 0.0045),

    )

    ypred = model.predict(raw.loc[valid_indices, features])
    raw.loc[valid_indices, 'k'] = ypred + 1
    raw.loc[valid_indices,'k'] = raw.loc[valid_indices,'k'] * raw.loc[valid_indices,'microbusiness_density']

    # Validate
    lastval = raw.loc[raw.dcount==TS, ['cfips', 'microbusiness_density']].set_index('cfips').to_dict()['microbusiness_density']
    dt = raw.loc[raw.dcount==TS, ['cfips', 'k']].set_index('cfips').to_dict()['k']
    
    df = raw.loc[raw.dcount==(TS+1), 
                 ['cfips', 'microbusiness_density', 'state', 'lastactive', 'mbd_lag_1']].reset_index(drop=True)
    df['pred'] = df['cfips'].map(dt)
    df['lastval'] = df['cfips'].map(lastval)
    
#     df.loc[df['lastval'].isnull(), 'lastval'] = df.loc[df['lastval'].isnull(), 'microbusiness_density']    
    
    df.loc[df['lastactive']<=ACT_THR, 'pred'] = df.loc[df['lastactive']<=ACT_THR, 'lastval']
    df.loc[df['cfips'].isin(values_list), 'pred'] = df.loc[df['cfips'].isin(values_list), 'lastval']
    
    raw.loc[raw.dcount==(TS+1), 'ypred'] = df['pred'].values
    raw.loc[raw.dcount==(TS+1), 'ypred_last'] = df['lastval'].values
        
    print(f'TS: {TS}')
    print('Last Value SMAPE:', smape(df['microbusiness_density'], df['lastval']) )
    print('SMAPE:', smape(df['microbusiness_density'], df['pred']))
    print()


ind = (raw.dcount > MONTH_1)&(raw.dcount <= MONTH_last)

print( 'SMAPE:', smape( raw.loc[ind, 'microbusiness_density'],  raw.loc[ind, 'ypred'] ) )
print( 'Last Value SMAPE:', smape( raw.loc[ind, 'microbusiness_density'],  raw.loc[ind, 'ypred_last'] ) )


39
TS: 39
Last Value SMAPE: 1.889206717018118
SMAPE: 1.8623186426836857

SMAPE: 1.8623186426836857
Last Value SMAPE: 1.889206717018118


In [20]:
raw.loc[train_indices, features]

,mbd_lag_1,act_lag_1,mbd_lag_2,act_lag_2,mbd_lag_3,act_lag_3,mbd_lag_4,act_lag_4,mbd_lag_5,act_lag_5,...,scale,K-means_Silhouette,K-means_Elbow,Elbow,rot_15_x,rot_15_y,rot_30_x,rot_30_y,rot_45_x,rot_45_y
19,0.012214,16.0,0.053902,83.0,-0.018422,70.0,-0.011802,55.0,-0.048653,-10.0,...,19,1,3,24,9.001702,53.851367,-15.14519,71.497710,-38.259962,84.271602
20,0.025641,34.0,0.012214,50.0,0.053902,117.0,-0.018422,104.0,-0.011802,89.0,...,20,1,3,24,9.001702,53.851367,-15.14519,71.497710,-38.259962,84.271602
21,0.000735,1.0,0.025641,35.0,0.012214,51.0,0.053902,118.0,-0.018422,105.0,...,21,1,3,24,9.001702,53.851367,-15.14519,71.497710,-38.259962,84.271602
22,-0.001470,-2.0,0.000735,-1.0,0.025641,33.0,0.012214,49.0,0.053902,116.0,...,22,1,3,24,9.001702,53.851367,-15.14519,71.497710,-38.259962,84.271602
23,-0.003679,-5.0,-0.001470,-7.0,0.000735,-6.0,0.025641,28.0,0.012214,44.0,...,23,1,3,24,9.001702,53.851367,-15.14519,71.497710,-38.259962,84.271602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147285,0.061798,11.0,-0.011111,9.0,-0.010989,7.0,-0.005465,6.0,0.000000,6.0,...,34,1,0,9,14.538139,70.278607,-15.81885,91.863464,-45.097811,107.187978
147286,0.031746,6.0,0.061798,17.0,-0.011111,15.0,-0.010989,13.0,-0.005465,12.0,...,35,1,0,9,14.538139,70.278607,-15.81885,91.863464,-45.097811,107.187978
147287,-0.005128,-1.0,0.031746,5.0,0.061798,16.0,-0.011111,14.0,-0.010989,12.0,...,36,1,0,9,14.538139,70.278607,-15.81885,91.863464,-45.097811,107.187978
147288,0.000000,0.0,-0.005128,-1.0,0.031746,5.0,0.061798,16.0,-0.011111,14.0,...,37,1,0,9,14.538139,70.278607,-15.81885,91.863464,-45.097811,107.187978


In [21]:
raw[raw['microbusiness_density'].isnull()]

,index,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest,lng,...,RNETMIG2021,rot_15_x,rot_15_y,rot_30_x,rot_30_y,rot_45_x,rot_45_y,ypred_last,ypred,k


In [22]:
TS = 40
print(TS)

model0 = get_model()

train_indices = (raw.istest==0) & (raw.dcount  < TS) & (raw.dcount >= 19) & (raw.lastactive>ACT_THR) & (~raw['cfips'].isin(values_list))
valid_indices = (raw.dcount == TS)
model0.fit(
    raw.loc[train_indices, features],
    raw.loc[train_indices, 'target'].clip(-0.0044, 0.0046),
)
ypred = model0.predict(raw.loc[valid_indices, features])
raw.loc[valid_indices, 'k'] = ypred + 1.
raw.loc[valid_indices,'k'] = raw.loc[valid_indices,'k'] * raw.loc[valid_indices,'microbusiness_density']

# Validate
lastval = raw.loc[raw.dcount==TS, ['cfips', 'microbusiness_density']].set_index('cfips').to_dict()['microbusiness_density']
dt = raw.loc[raw.dcount==TS, ['cfips', 'k']].set_index('cfips').to_dict()['k']

40


In [76]:
df = raw.loc[raw.dcount==(TS+1), ['cfips', 'microbusiness_density', 'state', 'lastactive', 'mbd_lag_1']].reset_index(drop=True)

In [77]:
df['pred'] = df['cfips'].map(dt)
df['lastval'] = df['cfips'].map(lastval)

df.loc[df['lastactive']<=ACT_THR, 'pred'] = df.loc[df['lastactive']<=ACT_THR, 'lastval']
df.loc[df['cfips'].isin(values_list), 'pred'] = df.loc[df['cfips'].isin(values_list), 'lastval']

raw.loc[raw.dcount==(TS+1), 'ypred'] = df['pred'].values
raw.loc[raw.dcount==(TS+1), 'ypred_last'] = df['lastval'].values

In [94]:
mask = (raw.lastactive > 150) & (raw.dcount==(TS+1)) & ((raw['Elbow']==0)|(raw['Elbow']==2)|(raw['Elbow']==4)|(raw['K-means_Elbow']==4)|(raw['K-means_Elbow']==6))
k = 0.5
raw.loc[mask, 'ypred'] = k * 1.005 * raw.loc[mask, 'ypred_last'] + (1-k) * raw.loc[mask, 'ypred'] 

In [95]:
raw.loc[mask]

,index,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest,lng,...,RNETMIG2021,rot_15_x,rot_15_y,rot_30_x,rot_30_y,rot_45_x,rot_45_y,ypred_last,ypred,k
41,NaN,1001_2023-01-01,1001,Autauga County,Alabama,2023-01-01,0.0,NaN,1,-86.642900,...,4.102668,9.001702,53.851367,-15.145190,71.497710,-38.259962,84.271602,3.470915,3.488061,1.0
88,NaN,1003_2023-01-01,1003,Baldwin County,Alabama,2023-01-01,0.0,NaN,1,-87.722745,...,28.969126,6.976482,52.385116,-17.250296,70.472449,-40.301494,83.757201,8.250630,8.291005,1.0
135,NaN,1005_2023-01-01,1005,Barbour County,Alabama,2023-01-01,0.0,NaN,1,-85.391068,...,-3.868858,8.683309,52.884978,-15.095226,70.295842,-37.845046,82.916160,1.252272,1.258479,1.0
182,NaN,1007_2023-01-01,1007,Bibb County,Alabama,2023-01-01,0.0,NaN,1,-87.126814,...,13.914989,9.323905,54.424063,-14.985975,72.140839,-38.274586,84.941337,1.287240,1.293539,1.0
229,NaN,1009_2023-01-01,1009,Blount County,Alabama,2023-01-01,0.0,NaN,1,-86.567006,...,2.404294,10.417811,55.228190,-13.855206,72.711800,-37.184013,85.240221,1.852060,1.861113,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146869,NaN,56025_2023-01-01,56025,Natrona County,Wyoming,2023-01-01,0.0,NaN,1,-106.798876,...,-8.135984,13.856465,69.139631,-16.193306,90.605570,-45.139530,105.896889,5.869197,5.897679,1.0
147057,NaN,56033_2023-01-01,56033,Sheridan County,Wyoming,2023-01-01,0.0,NaN,1,-106.879955,...,23.275119,15.601907,70.927042,-14.650110,92.229844,-43.903747,107.247334,238.060420,239.235613,1.0
147104,NaN,56035_2023-01-01,56035,Sublette County,Wyoming,2023-01-01,0.0,NaN,1,-109.914841,...,-3.444317,12.861584,69.757693,-17.920212,91.994629,-47.480776,107.962284,7.989996,8.028568,1.0
147198,NaN,56039_2023-01-01,56039,Teton County,Wyoming,2023-01-01,0.0,NaN,1,-110.589698,...,4.390265,13.814810,71.060250,-17.246401,93.343297,-47.132297,109.265153,26.314966,26.442994,1.0


In [104]:
mask = (raw.lastactive > 150) & (raw.dcount==(TS+1)) & ((raw['Elbow']==1)|(raw['Elbow']==5))
k = 0.5
raw.loc[mask, 'ypred'] = k * 0.995 * raw.loc[mask, 'ypred_last'] + (1-k) * raw.loc[mask, 'ypred'] 

In [105]:
raw.loc[mask]

,index,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,istest,lng,...,RNETMIG2021,rot_15_x,rot_15_y,rot_30_x,rot_30_y,rot_45_x,rot_45_y,ypred_last,ypred,k
4177,NaN,2220_2023-01-01,2220,Sitka City and Borough,Alaska,2023-01-01,0.0,NaN,1,-135.315175,...,-5.702406,20.261589,90.305878,-18.091548,117.223627,-55.211776,136.152779,6.183274,6.153191,1.0
4741,NaN,4012_2023-01-01,4012,La Paz County,Arizona,2023-01-01,0.0,NaN,1,-113.981207,...,3.273918,3.079360,62.080375,-27.780293,86.200914,-56.746766,104.447003,2.389446,2.377805,1.0
6621,NaN,5063_2023-01-01,5063,Independence County,Arkansas,2023-01-01,0.0,NaN,1,-91.569916,...,-3.147524,10.823537,58.223614,-14.831967,76.737949,-39.476697,90.022720,1.865136,1.856072,1.0
8313,NaN,5135_2023-01-01,5135,Sharp County,Arkansas,2023-01-01,0.0,NaN,1,-91.479284,...,29.223012,11.252503,58.605665,-14.423082,77.056202,-39.115758,90.255486,2.394376,2.382720,1.0
11039,NaN,6101_2023-01-01,6101,Sutter County,California,2023-01-01,0.0,NaN,1,-121.694619,...,-6.880122,6.207688,69.201458,-27.042314,94.652305,-58.449427,113.652754,3.714814,3.696728,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143673,NaN,55033_2023-01-01,55033,Dunn County,Wisconsin,2023-01-01,0.0,NaN,1,-91.896534,...,1.033115,19.630652,67.199798,-7.023241,84.873293,-33.198512,96.762813,2.824796,2.811053,1.0
144472,NaN,55067_2023-01-01,55067,Langlade County,Wisconsin,2023-01-01,0.0,NaN,1,-89.071936,...,7.704160,20.666414,66.773441,-5.337753,83.734183,-30.978162,94.988578,2.129450,2.119074,1.0
145553,NaN,55111_2023-01-01,55111,Sauk County,Wisconsin,2023-01-01,0.0,NaN,1,-89.948198,...,-0.076095,18.666637,65.227251,-7.365498,82.582700,-32.895687,94.310275,3.788552,3.770103,1.0
145694,NaN,55117_2023-01-01,55117,Sheboygan County,Wisconsin,2023-01-01,0.0,NaN,1,-87.945664,...,-0.144261,19.469315,64.993340,-6.109260,81.836404,-31.271498,93.102453,3.689452,3.671468,1.0


In [80]:
dt = raw.loc[raw.dcount==41, ['cfips', 'ypred']].set_index('cfips').to_dict()['ypred']
test = raw.loc[raw.istest==1, ['row_id', 'cfips','microbusiness_density']].copy()
test['microbusiness_density'] = test['cfips'].map(dt)
test = test[['row_id','microbusiness_density']]

In [81]:
sample_sub = pd.read_csv(BASE + 'revealed_test.csv')
sub_index = (sample_sub.first_day_of_month == '2022-11-01') | (sample_sub.first_day_of_month  == '2022-12-01')
test1 = pd.concat([sample_sub.loc[sub_index, :].drop([i for i  in sample_sub.columns if i !='row_id' and i != 'microbusiness_density'],axis=1).fillna(2), test])

In [82]:
test1 = test1.fillna(2)  # fill anything you want 

In [83]:
COLS = ['GEO_ID','NAME','S0101_C01_026E']
df2020 = pd.read_csv('./12/ACSST5Y2020.S0101-Data.csv',usecols=COLS, dtype = 'object')
df2020 = df2020.iloc[1:]
df2020['S0101_C01_026E'] = df2020['S0101_C01_026E'].astype('int')

In [84]:
df2021 = pd.read_csv('./12/ACSST5Y2021.S0101-Data.csv',usecols=COLS, dtype = 'object')
df2021 = df2021.iloc[1:]
df2021['S0101_C01_026E'] = df2021['S0101_C01_026E'].astype('int')

In [85]:
test1['cfips'] = test1.row_id.apply(lambda x: int(x.split('_')[0]))

In [86]:
kmeans=Kmean_df[['cfips','K-means_Silhouette','K-means_Elbow']]

In [87]:
kmeans.columns = ['cfips','Silhouette','Elbow']

In [88]:
Silhouette = kmeans.set_index('cfips').Silhouette.to_dict()
Elbow = kmeans.set_index('cfips').Elbow.to_dict()

In [89]:
df2020['cfips'] = df2020.GEO_ID.apply(lambda x: int(x.split('US')[-1]) )
adult2020 = df2020.set_index('cfips').S0101_C01_026E.to_dict()

df2021['cfips'] = df2021.GEO_ID.apply(lambda x: int(x.split('US')[-1]) )
adult2021 = df2021.set_index('cfips').S0101_C01_026E.to_dict()

test1['adult2020'] = test1.cfips.map(adult2020)
test1['adult2021'] = test1.cfips.map(adult2021)
test1['Silhouette'] = test1.cfips.map(Silhouette)
test1['Elbow'] = test1.cfips.map(Elbow)

In [90]:
test1.microbusiness_density = test1.microbusiness_density * test1.adult2020 / test1.adult2021

test1[['row_id','microbusiness_density']].to_csv('sub_testt.csv',index=False)

In [91]:
test1[3135*2:]

,row_id,microbusiness_density,cfips,adult2020,adult2021,Silhouette,Elbow
41,1001_2023-01-01,3.335031,1001,42496,44438,1,3
42,1001_2023-02-01,3.335031,1001,42496,44438,1,3
43,1001_2023-03-01,3.335031,1001,42496,44438,1,3
44,1001_2023-04-01,3.335031,1001,42496,44438,1,3
45,1001_2023-05-01,3.335031,1001,42496,44438,1,3
...,...,...,...,...,...,...,...
147340,56045_2023-02-01,1.836698,56045,5601,5499,1,6
147341,56045_2023-03-01,1.836698,56045,5601,5499,1,6
147342,56045_2023-04-01,1.836698,56045,5601,5499,1,6
147343,56045_2023-05-01,1.836698,56045,5601,5499,1,6
